In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports
import pandas as pd
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import matplotlib.pyplot as plt
import seaborn as sns


import multiprocessing as mp   
import os
import time
import psutil
import datetime

from IPython.display import display
from sklearn import metrics

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


PATH = os.getcwd() # '/Users/admin/Documents/Projects/unit2'

# Import Data

# Select Columns to import
columns = ['Created Date', 'Closed Date', 'Due Date', 'Location Type',
           'Incident Zip', 'City', 'Status', 'Open Data Channel Type',
           'Descriptor', 'Latitude', 'Longitude']

# Rename Columns
# names = ['created_date', 'closed_date', 'descriptor',
#         'location_type', 'incident_zip', 'city', 'status', 
#         'open_data_channel_type','latitude', 'longitude']
# http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/fixing-column-names-in-pandas/
# names=names

# Converts Created and Closed Date to Datetime format
dates = ['Created Date', 'Closed Date', 'Due Date']

df_nyc = pd.read_csv(f'{PATH}/data/311_Service_requests_2018.csv', 
                    usecols=columns, parse_dates=dates, 
                    infer_datetime_format=True, nrows=100000, low_memory=False)

# Display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
            
# Check the number of cores and memory usage
num_cores = mp.cpu_count()
print("This kernel has ",num_cores,"cores and you can find the information regarding the memory usage:",psutil.virtual_memory())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This kernel has  24 cores and you can find the information regarding the memory usage: svmem(total=17179869184, available=7247560704, percent=57.8, used=8970964992, free=167849984, active=7219859456, inactive=7078305792, wired=1751105536)


In [5]:
df_nyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
Created Date              100000 non-null datetime64[ns]
Closed Date               98520 non-null datetime64[ns]
Descriptor                98781 non-null object
Location Type             83263 non-null object
Incident Zip              97080 non-null object
City                      97084 non-null object
Status                    100000 non-null object
Due Date                  25043 non-null datetime64[ns]
Open Data Channel Type    100000 non-null object
Latitude                  95069 non-null float64
Longitude                 95069 non-null float64
dtypes: datetime64[ns](3), float64(2), object(6)
memory usage: 8.4+ MB


In [6]:
df_nyc.dtypes

Created Date              datetime64[ns]
Closed Date               datetime64[ns]
Descriptor                        object
Location Type                     object
Incident Zip                      object
City                              object
Status                            object
Due Date                  datetime64[ns]
Open Data Channel Type            object
Latitude                         float64
Longitude                        float64
dtype: object

In [7]:
display_all(df_nyc)

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Due Date,Open Data Channel Type,Latitude,Longitude
0,2018-01-01 00:00:00,2018-01-10 15:53:17,Condition Attracting Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,2018-01-31 01:51:21,MOBILE,40.631571,-74.147439
1,2018-01-01 00:00:00,2018-01-18 00:00:00,Rat Sighting,3+ Family Apt. Building,11216,BROOKLYN,Closed,2018-01-31 02:12:01,ONLINE,40.691126,-73.945449
2,2018-01-01 00:00:00,2018-01-10 13:56:35,Signs of Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,2018-01-31 10:20:58,PHONE,40.632714,-74.156395
3,2018-01-01 00:00:00,2018-01-09 00:00:00,Mouse Sighting,3+ Family Apt. Building,10022,NEW YORK,Closed,2018-01-31 20:39:14,MOBILE,40.755254,-73.965699
4,2018-01-01 00:00:00,2018-01-22 00:00:00,Rat Sighting,Other (Explain Below),10467,BRONX,Closed,2018-01-31 15:12:21,MOBILE,40.880459,-73.876574
5,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,11225,BROOKLYN,Closed,2018-01-31 05:35:46,PHONE,40.670086,-73.954767
6,2018-01-01 00:00:00,2018-01-26 11:57:11,Rat Sighting,3+ Family Apt. Building,11385,Ridgewood,Closed,2018-01-31 19:49:25,PHONE,40.701538,-73.907788
7,2018-01-01 00:00:00,2018-01-09 00:00:00,Dog,3+ Family Apartment Building,11435,Jamaica,Closed,2018-01-31 11:40:44,UNKNOWN,40.709597,-73.814789
8,2018-01-01 00:00:00,2018-01-09 00:00:00,Condition Attracting Rodents,3+ Family Apt. Building,10457,BRONX,Closed,2018-01-31 09:31:49,ONLINE,40.840838,-73.910309
9,2018-01-01 00:00:00,2018-01-03 00:00:00,Signs of Rodents,3+ Family Apt. Building,10040,NEW YORK,Closed,2018-01-31 10:15:26,PHONE,40.855888,-73.931570


In [8]:
df_nyc.shape

(100000, 11)

In [9]:
df_nyc.isnull().sum()

Created Date                  0
Closed Date                1480
Descriptor                 1219
Location Type             16737
Incident Zip               2920
City                       2916
Status                        0
Due Date                  74957
Open Data Channel Type        0
Latitude                   4931
Longitude                  4931
dtype: int64

In [71]:
df_nyc.dtypes

Created Date               datetime64[ns]
Closed Date                datetime64[ns]
Descriptor                         object
Location Type                      object
Incident Zip                       object
City                               object
Status                             object
Open Data Channel Type             object
Latitude                          float64
Longitude                         float64
total_time                timedelta64[ns]
dtype: object

In [16]:
# Drop Nulls

null_cols = ['Closed Date', 'Due Date', 'Descriptor', 'Location Type', 'Incident Zip',
             'City', 'Latitude', 'Longitude']

def drop_nulls(df_nyc):
    for cols in null_cols:
        df_nyc = df_nyc.dropna(subset=[null_cols], inplace=True)
        
    return df_nyc

df_nyc = drop_nulls(df_nyc)

KeyError: ['Closed Date', 'Due Date', 'Descriptor', 'Location Type', 'Incident Zip', 'City', 'Latitude', 'Longitude']

In [17]:
# Drop Nulls
df_nyc.dropna(subset=['Location Type'], inplace=True)
df_nyc.dropna(subset=['Latitude'], inplace=True)
df_nyc.dropna(subset=['Longitude'], inplace=True)
df_nyc.dropna(subset=['Descriptor'], inplace=True)
df_nyc.dropna(subset=['Closed Date'], inplace=True) 
df_nyc.dropna(subset=['Incident Zip'], inplace=True)
df_nyc.dropna(subset=['Due Date'], inplace=True) 

In [18]:
df_nyc.isnull().sum()

Created Date              0
Closed Date               0
Descriptor                0
Location Type             0
Incident Zip              0
City                      0
Status                    0
Due Date                  0
Open Data Channel Type    0
Latitude                  0
Longitude                 0
dtype: int64

In [50]:
df_nyc['Location Type'].value_counts()

RESIDENTIAL BUILDING            51315
Street/Sidewalk                  9713
Sidewalk                         6147
Residential Building/House       5809
Street                           3667
Property Address                  817
Store/Commercial                  673
Building (Non-Residential)        554
Senior Address                    417
Club/Bar/Restaurant               395
3+ Family Apt. Building           381
NYC Street Address                332
Park                              302
Restaurant/Bar/Deli/Bakery        279
1-, 2- and 3- Family Home         255
Mixed Use                         200
School                            155
Bus Stop Shelter                  145
Other                             139
3+ Family Apartment Building      137
Comercial                         128
1-2 Family Dwelling               126
Other (Explain Below)             112
Residential Building              112
Co-Op Unit                         97
Highway                            71
Lot         

In [48]:
df_nyc['Descriptor'].value_counts()

ENTIRE BUILDING                                         21655
APARTMENT ONLY                                          13853
Loud Music/Party                                         3718
No Access                                                3544
E9 Snow / Icy Sidewalk                                   2354
Banging/Pounding                                         2193
Request Large Bulky Item Collection                      2108
No Water (WNW)                                           1478
Street Light Out                                         1394
WATER SUPPLY                                             1355
Leak (Use Comments) (WA2)                                1241
PESTS                                                    1179
WINDOW FRAME                                             1076
Blocked Hydrant                                          1075
CEILING                                                  1014
WALL                                                     1006
Pothole 

In [19]:
df_nyc['total_time'] = df_nyc['Closed Date'] - df_nyc['Created Date']

In [20]:
df_nyc['late'] = df_nyc['Closed Date'] > df_nyc['Due Date']

In [21]:
df_nyc.dtypes

Created Date               datetime64[ns]
Closed Date                datetime64[ns]
Descriptor                         object
Location Type                      object
Incident Zip                       object
City                               object
Status                             object
Due Date                   datetime64[ns]
Open Data Channel Type             object
Latitude                          float64
Longitude                         float64
total_time                timedelta64[ns]
late                                 bool
dtype: object

In [22]:
df_nyc.head()

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Due Date,Open Data Channel Type,Latitude,Longitude,total_time,late
0,2018-01-01,2018-01-10 15:53:17,Condition Attracting Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,2018-01-31 01:51:21,MOBILE,40.631571,-74.147439,9 days 15:53:17,False
1,2018-01-01,2018-01-18 00:00:00,Rat Sighting,3+ Family Apt. Building,11216,BROOKLYN,Closed,2018-01-31 02:12:01,ONLINE,40.691126,-73.945449,17 days 00:00:00,False
2,2018-01-01,2018-01-10 13:56:35,Signs of Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,2018-01-31 10:20:58,PHONE,40.632714,-74.156395,9 days 13:56:35,False
3,2018-01-01,2018-01-09 00:00:00,Mouse Sighting,3+ Family Apt. Building,10022,NEW YORK,Closed,2018-01-31 20:39:14,MOBILE,40.755254,-73.965699,8 days 00:00:00,False
4,2018-01-01,2018-01-22 00:00:00,Rat Sighting,Other (Explain Below),10467,BRONX,Closed,2018-01-31 15:12:21,MOBILE,40.880459,-73.876574,21 days 00:00:00,False


In [54]:
df_nyc['total_time'].mean()

Timedelta('9 days 18:05:17.543392')

In [63]:
df_nyc['total_time'].describe()

count                      98520
mean      9 days 18:05:17.543392
std      27 days 14:08:53.724728
min          -366 days +23:59:00
25%              0 days 08:24:12
50%              3 days 05:21:10
75%              7 days 11:46:00
max            624 days 16:15:19
Name: total_time, dtype: object

In [99]:
df_nyc.dtypes

Created Date               datetime64[ns]
Closed Date                datetime64[ns]
Descriptor                         object
Location Type                      object
Incident Zip                       object
City                               object
Status                             object
Open Data Channel Type             object
Latitude                          float64
Longitude                         float64
total_time                timedelta64[ns]
dtype: object

In [119]:
# https://stackoverflow.com/questions/52939673/pandas-and-datetime-typeerror-cannot-compare-a-timedeltaindex-with-type-float

mask_days = df_nyc[df_nyc['total_time'] > pd.Timedelta(300,'D')]

In [120]:
mask_days

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Open Data Channel Type,Latitude,Longitude,total_time
3228,2018-01-01 10:54:12,2018-11-07 15:03:26,Planted More Than 2 Years Ago,Street,11355,FLUSHING,Closed,OTHER,40.760130,-73.819551,310 days 04:09:14
3235,2018-01-01 10:54:24,2018-11-07 15:03:27,Planted More Than 2 Years Ago,Street,11355,FLUSHING,Closed,OTHER,40.760130,-73.819551,310 days 04:09:03
5100,2018-01-01 14:13:00,2019-05-22 12:00:00,14B Derelict Bicycle,Street,11201,BROOKLYN,Closed,ONLINE,40.694026,-73.991565,505 days 21:47:00
10016,2018-01-02 04:23:24,2019-05-30 00:00:00,Graffiti,Residential,11221,BROOKLYN,Closed,UNKNOWN,40.687294,-73.938886,512 days 19:36:36
11844,2018-01-02 09:31:30,2018-11-07 15:03:30,Trees and Sidewalks Program,Street,11385,RIDGEWOOD,Closed,PHONE,40.708271,-73.918744,309 days 05:32:00
12747,2018-01-02 10:28:12,2019-01-23 16:01:10,For One Address,Street,11102,ASTORIA,Closed,OTHER,40.768877,-73.922562,386 days 05:32:58
13602,2018-01-02 11:25:51,2019-05-09 07:30:29,Hitting Building,Street,11223,BROOKLYN,Closed,OTHER,40.606807,-73.971430,491 days 20:04:38
14415,2018-01-02 12:17:30,2018-11-07 15:03:33,Trees and Sidewalks Program,Street,11436,JAMAICA,Closed,OTHER,40.673097,-73.798462,309 days 02:46:03
14647,2018-01-02 12:33:24,2018-11-07 15:04:38,Planted Less Than 2 Years Ago,Street,11367,FLUSHING,Closed,PHONE,40.723682,-73.821417,309 days 02:31:14
14958,2018-01-02 12:57:41,2018-11-07 15:04:38,Planted More Than 2 Years Ago,Street,11362,LITTLE NECK,Closed,PHONE,40.753957,-73.728513,309 days 02:06:57


In [122]:
mask_zero = df_nyc[df_nyc['total_time'] == pd.Timedelta(0, 'm')]

In [123]:
# I want to exlude these. Its either bad data or error or both. 
# At the moment I lack the information to process these. 
mask_zero

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Open Data Channel Type,Latitude,Longitude,total_time
5,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,11225,BROOKLYN,Closed,PHONE,40.670086,-73.954767,0 days
17,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,10463,BRONX,Closed,PHONE,40.881582,-73.898989,0 days
23,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,Other (Explain Below),11206,BROOKLYN,Closed,MOBILE,40.694006,-73.946067,0 days
2575,2018-01-01 09:44:09,2018-01-01 09:44:09,SIGNAGE MISSING,RESIDENTIAL BUILDING,11206,BROOKLYN,Closed,PHONE,40.694387,-73.944812,0 days
8761,2018-01-01 22:12:27,2018-01-01 22:12:27,SIGNAGE MISSING,RESIDENTIAL BUILDING,11218,BROOKLYN,Closed,ONLINE,40.636432,-73.970124,0 days
8798,2018-01-01 22:17:08,2018-01-01 22:17:08,SIGNAGE MISSING,RESIDENTIAL BUILDING,11218,BROOKLYN,Closed,ONLINE,40.636432,-73.970124,0 days
9328,2018-01-02 00:00:00,2018-01-02 00:00:00,Rat Sighting,1-2 Family Dwelling,11238,BROOKLYN,Closed,MOBILE,40.678808,-73.970332,0 days
9331,2018-01-02 00:00:00,2018-01-02 00:00:00,Condition Attracting Rodents,3+ Family Apt. Building,11221,BROOKLYN,Closed,PHONE,40.686541,-73.928286,0 days
9332,2018-01-02 00:00:00,2018-01-02 00:00:00,Condition Attracting Rodents,3+ Family Apt. Building,10022,NEW YORK,Closed,PHONE,40.760157,-73.969379,0 days
9345,2018-01-02 00:00:00,2018-01-02 00:00:00,Rat Sighting,3+ Family Apt. Building,10453,BRONX,Closed,PHONE,40.848057,-73.914879,0 days


In [130]:
# Can I exclude anything less than 1 day? 
mask_less = df_nyc[df_nyc['total_time'] < pd.Timedelta(1, 'D')]

In [131]:
mask_less

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Open Data Channel Type,Latitude,Longitude,total_time
5,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,11225,BROOKLYN,Closed,PHONE,40.670086,-73.954767,00:00:00
17,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,10463,BRONX,Closed,PHONE,40.881582,-73.898989,00:00:00
23,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,Other (Explain Below),11206,BROOKLYN,Closed,MOBILE,40.694006,-73.946067,00:00:00
38,2018-01-01 00:01:23,2018-01-01 02:13:48,No Access,Street/Sidewalk,11225,BROOKLYN,Closed,PHONE,40.660543,-73.943589,02:12:25
42,2018-01-01 00:04:05,2018-01-01 13:05:25,Loud Music/Party,Residential Building/House,11206,BROOKLYN,Closed,MOBILE,40.696189,-73.942217,13:01:20
43,2018-01-01 00:04:17,2018-01-01 20:33:07,No Access,Street/Sidewalk,11385,RIDGEWOOD,Closed,PHONE,40.701975,-73.902003,20:28:50
44,2018-01-01 00:04:34,2018-01-01 13:45:45,Loud Music/Party,Residential Building/House,11378,MASPETH,Closed,PHONE,40.721666,-73.902147,13:41:11
45,2018-01-01 00:04:51,2018-01-01 00:44:05,Loud Music/Party,Residential Building/House,11432,JAMAICA,Closed,PHONE,40.709136,-73.780885,00:39:14
46,2018-01-01 00:05:06,2018-01-01 00:59:21,Banging/Pounding,Store/Commercial,11222,BROOKLYN,Closed,MOBILE,40.729797,-73.958385,00:54:15
47,2018-01-01 00:05:16,2018-01-01 01:19:46,Loud Music/Party,Residential Building/House,11355,FLUSHING,Closed,ONLINE,40.750501,-73.823002,01:14:30


In [132]:
mask_less.describe()

,Latitude,Longitude,total_time
count,20887.000000,20887.000000,20887
mean,40.732840,-73.918679,0 days 05:45:39.163546
std,0.087965,0.078241,0 days 06:09:53.280497
min,40.502305,-74.252075,0 days 00:00:00
25%,40.667404,-73.959998,0 days 01:21:09
50%,40.723432,-73.920771,0 days 03:18:17
75%,40.817793,-73.869555,0 days 07:39:15.500000
max,40.911102,-73.700837,0 days 23:59:57


In [139]:
df_nyc = df_nyc[df_nyc['total_time'] > pd.Timedelta(1, 'm')]

In [140]:
df_nyc

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Open Data Channel Type,Latitude,Longitude,total_time
0,2018-01-01 00:00:00,2018-01-10 15:53:17,Condition Attracting Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,MOBILE,40.631571,-74.147439,9 days 15:53:17
1,2018-01-01 00:00:00,2018-01-18 00:00:00,Rat Sighting,3+ Family Apt. Building,11216,BROOKLYN,Closed,ONLINE,40.691126,-73.945449,17 days 00:00:00
2,2018-01-01 00:00:00,2018-01-10 13:56:35,Signs of Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,PHONE,40.632714,-74.156395,9 days 13:56:35
3,2018-01-01 00:00:00,2018-01-09 00:00:00,Mouse Sighting,3+ Family Apt. Building,10022,NEW YORK,Closed,MOBILE,40.755254,-73.965699,8 days 00:00:00
4,2018-01-01 00:00:00,2018-01-22 00:00:00,Rat Sighting,Other (Explain Below),10467,BRONX,Closed,MOBILE,40.880459,-73.876574,21 days 00:00:00
6,2018-01-01 00:00:00,2018-01-26 11:57:11,Rat Sighting,3+ Family Apt. Building,11385,Ridgewood,Closed,PHONE,40.701538,-73.907788,25 days 11:57:11
7,2018-01-01 00:00:00,2018-01-09 00:00:00,Dog,3+ Family Apartment Building,11435,Jamaica,Closed,UNKNOWN,40.709597,-73.814789,8 days 00:00:00
8,2018-01-01 00:00:00,2018-01-09 00:00:00,Condition Attracting Rodents,3+ Family Apt. Building,10457,BRONX,Closed,ONLINE,40.840838,-73.910309,8 days 00:00:00
9,2018-01-01 00:00:00,2018-01-03 00:00:00,Signs of Rodents,3+ Family Apt. Building,10040,NEW YORK,Closed,PHONE,40.855888,-73.931570,2 days 00:00:00
10,2018-01-01 00:00:00,2018-01-03 00:00:00,Mouse Sighting,3+ Family Apt. Building,11102,Astoria,Closed,PHONE,40.768979,-73.930988,2 days 00:00:00


In [90]:
mask = df_nyc['Status'].str.contains('Closed')

In [91]:
df_nyc[mask]

,Created Date,Closed Date,Descriptor,Location Type,Incident Zip,City,Status,Open Data Channel Type,Latitude,Longitude,total_time
0,2018-01-01 00:00:00,2018-01-10 15:53:17,Condition Attracting Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,MOBILE,40.631571,-74.147439,9 days 15:53:17
1,2018-01-01 00:00:00,2018-01-18 00:00:00,Rat Sighting,3+ Family Apt. Building,11216,BROOKLYN,Closed,ONLINE,40.691126,-73.945449,17 days 00:00:00
2,2018-01-01 00:00:00,2018-01-10 13:56:35,Signs of Rodents,1-2 Family Dwelling,10303,STATEN ISLAND,Closed,PHONE,40.632714,-74.156395,9 days 13:56:35
3,2018-01-01 00:00:00,2018-01-09 00:00:00,Mouse Sighting,3+ Family Apt. Building,10022,NEW YORK,Closed,MOBILE,40.755254,-73.965699,8 days 00:00:00
4,2018-01-01 00:00:00,2018-01-22 00:00:00,Rat Sighting,Other (Explain Below),10467,BRONX,Closed,MOBILE,40.880459,-73.876574,21 days 00:00:00
5,2018-01-01 00:00:00,2018-01-01 00:00:00,Rat Sighting,3+ Family Apt. Building,11225,BROOKLYN,Closed,PHONE,40.670086,-73.954767,0 days 00:00:00
6,2018-01-01 00:00:00,2018-01-26 11:57:11,Rat Sighting,3+ Family Apt. Building,11385,Ridgewood,Closed,PHONE,40.701538,-73.907788,25 days 11:57:11
7,2018-01-01 00:00:00,2018-01-09 00:00:00,Dog,3+ Family Apartment Building,11435,Jamaica,Closed,UNKNOWN,40.709597,-73.814789,8 days 00:00:00
8,2018-01-01 00:00:00,2018-01-09 00:00:00,Condition Attracting Rodents,3+ Family Apt. Building,10457,BRONX,Closed,ONLINE,40.840838,-73.910309,8 days 00:00:00
9,2018-01-01 00:00:00,2018-01-03 00:00:00,Signs of Rodents,3+ Family Apt. Building,10040,NEW YORK,Closed,PHONE,40.855888,-73.931570,2 days 00:00:00


In [92]:
df_nyc = df_nyc[mask]

In [141]:
df_nyc.shape

(77599, 11)

In [142]:
df_nyc['total_time'].describe()

count                      77599
mean      9 days 08:39:27.022203
std      22 days 06:56:20.421366
min              0 days 00:01:06
25%              0 days 20:18:21
50%              3 days 22:30:42
75%       7 days 16:51:20.500000
max            613 days 11:57:57
Name: total_time, dtype: object

In [144]:
df_nyc.isnull().sum()

Created Date              0
Closed Date               0
Descriptor                0
Location Type             0
Incident Zip              0
City                      0
Status                    0
Open Data Channel Type    0
Latitude                  0
Longitude                 0
total_time                0
dtype: int64

###  Models

In [151]:
df_nyc['created_date']= pd.to_datetime(df_nyc['created_date'])
df_nyc['created_date_year'] = df_nyc['created_date'].dt.year
df_nyc['created_date_month'] = df_nyc['created_date'].dt.month
df_nyc['created_date_day'] = df_nyc['created_date'].dt.day
df_nyc['created_date_hour'] = df_nyc['created_date'].dt.hour
df_nyc['created_date_day_of_week'] = df_nyc['created_date'].dt.dayofweek

df_nyc.drop('created_date',axis=1,inplace=True)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [146]:
# Baseline

# Determine majority class
y_train = df_nyc['total_time']
y_train.value_counts(normalize=True)

8 days 00:00:00      0.000490
7 days 00:00:00      0.000387
5 days 00:00:00      0.000309
9 days 00:00:00      0.000284
6 days 00:00:00      0.000271
1 days 00:00:00      0.000258
4 days 00:00:00      0.000245
2 days 00:00:00      0.000245
13 days 00:00:00     0.000232
1 days 00:00:01      0.000193
12 days 00:00:00     0.000193
3 days 00:00:00      0.000180
10 days 00:00:00     0.000168
11 days 00:00:00     0.000168
14 days 00:00:00     0.000142
41 days 19:48:14     0.000129
15 days 23:18:07     0.000129
18 days 23:20:32     0.000129
33 days 23:34:37     0.000129
31 days 14:05:08     0.000129
59 days 21:15:06     0.000129
15 days 11:32:58     0.000116
16 days 23:02:37     0.000116
9 days 20:39:27      0.000116
9 days 04:19:32      0.000116
33 days 01:30:15     0.000116
15 days 21:45:13     0.000116
11 days 19:04:43     0.000116
7 days 19:19:35      0.000116
7 days 12:59:08      0.000116
                       ...   
15 days 00:20:55     0.000013
0 days 06:22:38      0.000013
7 days 02:

In [148]:
# What if we guessed the majority class for every prediction?
majority_class = y_train.mean()
y_pred = [majority_class] * len(y_train)
print(len(y_pred))

77599


In [149]:
# What is the baseline accuracy if we guessed the majority class for every prediction?

# Accuracy of majority class baseline = frequency of majority class
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

ValueError: Classification metrics can't handle a mix of multiclass and unknown targets

In [ ]:
train.shape

In [ ]:
# The status_group column is the target
target = 'time_until_resolved'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [ ]:
# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, random_state=42)

In [ ]:
train.shape, val.shape

In [ ]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
# Split train into train & val. Make val the same size as test.
target = 'time_until_resolved'
train, val = train_test_split(train, test_size=len(test),  
                              stratify=train[target], random_state=42)

# Wrangle train, validate, and test sets in the same way
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)


# Arrange data into X features matrix and y target vector
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

# Make pipeline!
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))

In [ ]:
X_train = train
y_train = train['time_until_resolved']

X_train.shape, y_train.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.80, test_size=0.20, 
    stratify=y_train, random_state=42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape